In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys

code_dir = os.path.expandvars('$OSCBAGDIS_DATAPROC_CODE')
sys.path.append(code_dir)

import mne
import utils  #my code
import json
import matplotlib.pyplot as plt
import numpy as np
import h5py
import utils_preproc as upre
import gc; gc.collect()

subdir = ''
if os.environ.get('DATA_DUSS') is not None:
    data_dir_input = os.path.expandvars('$DATA_DUSS')   + subdir
    data_dir_output = os.path.expandvars('$DATA_DUSS')
else:
    data_dir_input = '/home/demitau/data'   + subdir
    data_dir_output = '/home/demitau/data' 
    

read_resampled = 1

    
subjinds = np.arange(1,11)
subjinds = [1,2,3,4]
subjinds = [1,2,4,5,7]
subjinds = [1,2]
tasks = ['hold' , 'move', 'rest']
medstates = ['off','on']


fnames_noext = []
for subjind in subjinds:
    sis = '{:02d}'.format(subjind)
    for medstate in medstates:
        for task in tasks:
            addStr = ''

        #fnames_noext = ['S{}_off_{}'.format(sis,task), 'S{}_on_{}'.format(sis,task)]
        #fnames_noext = ['S01_off_hold', 'S01_on_hold']
        #fnames_noext = ['S01_off_move', 'S01_on_move']
        #fnames_noext = ['S02_off_move', 'S02_on_move']
            fname_noext = 'S{}_{}_{}'.format(sis,medstate,task)
            #addStr += '_resample'
            addStr += '_resample_notch_highpass'

            
#             fname = fname_noext + '.mat'
#             print('Reading {}'.format(fname) )


            if read_resampled:
                fname_full = os.path.join(data_dir_output,fname_noext + addStr + '_raw.fif')
            else:
                fname_full = os.path.join(data_dir_input,fname)
            
            #print(fname_full)
            if os.path.exists(fname_full):
                fnames_noext += [fname_noext]
                
#                 print('Warning: path does not exist!, skip! {}'.format(fname_full))
#                 continue



#rawname = rawname_ + '_resample_raw.fif'
#fname_full = os.path.join(data_dir,rawname)
    
# read file -- resampled to 256 Hz,  Electa MEG, EMG, LFP, EOG channels
#raw = mne.io.read_raw_fif(fname_full, None)

print (fnames_noext)

In [ ]:
highpass = None
#highpass = 2

## Reading subraws

In [ ]:
# modalities = ['EMG','LFP']
# mod_picks = ['EMG.*old', 'LFP.*']

# subraws = {}
# for mod, picks in zip(modalities, mod_picks):
#     subraws[mod] = []
    
# for rawname_naked in fnames_noext: 
#     raw = uprep.getRaw(rawname_naked)
#     raw.load_data()
#     for mod, picks in zip(modalities, mod_picks):
#         subraw = uprep.getSubRaw(rawname_naked, raw = raw, picks = picks)
#         if highpass is not None:
#             subraw.filter(l_freq=2, h_freq=None, picks='all')
#         if subraw is not None:
#             subraws[mod] += [subraw]

# gc.collect()

In [ ]:
print('highpass=',highpass)

In [ ]:
fnames_noext

In [ ]:
import utils_preproc as upre

#modalities = ['EMG','LFP','MEG'] #,'src']
modalities = ['src'] #,'src']
rawnames = fnames_noext
raws_permod_both_sides = upre.loadRaws(rawnames,modalities, 'parcel_aal', 
                                       'parcel_ICA', 10, use_saved=False,
                                      highpass_lfreq=highpass)

gc.collect()

In [ ]:
# check channel counts agree
for mod in modalities:   
    n_channels_all = [ len(raws_permod_both_sides[rn][mod].ch_names) for rn in rawnames ] 
    unch =  np.unique(n_channels_all)
    assert len(unch ) == 1, unch


In [ ]:
sources_type = 'parcel_aal'
src_file_grouping_ind = 10
src_grouping = 0

In [ ]:
src_rec_info_fn = '{}_{}_grp{}_src_rec_info'.format(rawname_,sources_type,src_file_grouping_ind)
src_rec_info_fn_full = os.path.join(gv.data_dir, src_rec_info_fn + '.npz')
rec_info = np.load(src_rec_info_fn_full, allow_pickle=True)

roi_labels = rec_info['label_groups_dict'][()]      # dict of (orderd) lists
srcgrouping_names_sorted = rec_info['srcgroups_key_order'][()]  # order of grouping names
assert set(roi_labels.keys() ) == set(srcgrouping_names_sorted )
assert len(roi_labels) == 1, 'several groupings in single run -- not implmemented'
# assuming we have only one grouping present
roi_labels_cur = roi_labels[srcgrouping_names_sorted[src_grouping ]  ]


In [ ]:
# rescale LFP recordings at oen of the sides only according to the rest scale
if 'LFP' in modalities:
    upre.rescaleRaws(raws_permod_both_sides, mod='LFP')
if 'src' in modalities:
    upre.rescaleRaws(raws_permod_both_sides, mod='src',combine_within_medcond=True,
                    roi_labels=roi_labels, srcgrouping_names_sorted=srcgrouping_names_sorted)

In [ ]:
meg_chns = raws_permod_both_sides['S01_off_hold']['MEG'].ch_names
nmegs_to_show = 10
meg_chis = np.random.choice(len(meg_chns),nmegs_to_show)

In [ ]:
src_chns = raws_permod_both_sides['S01_off_hold']['src'].ch_names
nsrcs_to_show = 10
src_chis = np.random.choice(len(src_chns),nsrcs_to_show)

In [ ]:
vals = np.linspace(0,1, 15)
np.random.shuffle(vals)
cmap = plt.cm.colors.ListedColormap(plt.cm.jet(vals))

# Over intervals

In [ ]:
int_types_templ = ['trem_{}', 'notrem_{}', 'hold_{}', 'move_{}']

In [ ]:
import globvars as gv

sfreq = raws_permod_both_sides[rawnames[0]][modalities[0]].info['sfreq']
import utils_tSNE as utsne
print(sfreq)

## Collect stats

In [ ]:
#%debug
#side_to_use = 'main_trem'
side_to_use =  'main_move'
qmult = 1.15  # how much to multiply the qunatile span
qch_hist_xshift = 0.1

xshifts_rel_perint_permod = {}  # set shifts for displaying so that histograms don't interesect
dat_permod_perraw_perint = {}
for mod in modalities:
    dat_permod_perraw_perint[mod] = {}
    xshifts_rel_perint = {}
    
    
    n_channels_all = [ len(raws_permod_both_sides[rn][mod].ch_names) for rn in rawnames ] 
    nmn,nmx = np.min(n_channels_all), np.max(n_channels_all)
    assert nmn==nmx, (nmn,nmx)
    n_channels = n_channels_all[0]
    for int_type in int_types_templ:
        xshifts_rel = [0]*n_channels
        xshifts_rel_perint[int_type] = xshifts_rel

    # first gather info 
    for i in range(len(fnames_noext)):
        rawind = i
        rawname_ = fnames_noext[rawind]
        dat_permod_perraw_perint[mod][rawname_] = {}
        subj,medcond,task  = utils.getParamsFromRawname(rawname_)
        #raw = subraws[mod ][rawind]
        raw = raws_permod_both_sides[rawname_][mod]
        
        #fname_full_LFPartif = os.path.join(gv.data_dir, '{}_ann_LFPartif.txt'.format(rawname_) )
        #anns_LFP_artif = mne.read_annotations(fname_full_LFPartif)
        
        anns_mvt, anns_artif_pri, times2, dataset_bounds = \
        utsne.concatAnns([rawname_],[raw.times] )
        ivalis_mvt = utils.ann2ivalDict(anns_mvt)
        ivalis_mvt_tb, ivalis_mvt_tb_indarrays = utsne.getAnnBins(ivalis_mvt, raw.times,
                                                                    0, sfreq, 1, 1, 
                                                                      dataset_bounds)
        ivalis_mvt_tb_indarrays_merged = utsne.mergeAnnBinArrays(ivalis_mvt_tb_indarrays)

        
        if mod == 'LFP':
            prefixes = ['_ann_LFPartif']
        elif mod in ['MEG','src']:
            prefixes = ['_ann_MEGartif']
        anns_artif, anns_artif_pri, times2, dataset_bounds = \
        utsne.concatAnns([rawname_],[raw.times],prefixes )
        ivalis_artif = utils.ann2ivalDict(anns_artif)
        ivalis_artif_tb, ivalis_artif_tb_indarrays = utsne.getAnnBins(ivalis_artif, raw.times,
                                                                    0, sfreq, 1, 1, 
                                                                      dataset_bounds)
        ivalis_artif_tb_indarrays_merged = utsne.mergeAnnBinArrays(ivalis_artif_tb_indarrays)
        

        for j in range(len(int_types_templ)):
            if mod == 'MEG':
                chdata,times = raw[meg_chis,:]
                chnames = np.array(raw.ch_names)[meg_chis]
            elif mod == 'src':
                chdata,times = raw[src_chis,:]
                chnames = np.array(raw.ch_names)[src_chis]
            else:
                chdata = raw.get_data()
                chnames = raw.ch_names
                    
            side = None
            if side_to_use == 'main_trem':
                side = gv.gen_subj_info[subj]['tremor_side']                
            elif side_to_use == 'main_move':
                side = gv.gen_subj_info[subj].get('move_side',None)
            if side is None:
                print('{}: {} is None'.format(rawname_, side_to_use))
            side_letter = side[0].upper()
            
            itcur = int_types_templ[j]
            int_type_cur = itcur.format(side_letter)

            ivalbins = ivalis_mvt_tb_indarrays_merged.get(int_type_cur, None )
            if ivalbins is None:
                continue
            mask = np.zeros(chdata.shape[1], dtype=bool)
            mask[ivalbins] = True
            dat_permod_perraw_perint[mod][rawname_][itcur] = [0]*len(chnames) #chds
                
            for chni,chn in enumerate(chnames):
                artif_bins_cur = ivalis_artif_tb_indarrays_merged.get('BAD_{}'.format(chn),[])
                mbefore = np.sum(mask)
                mask[artif_bins_cur] = False
                mafter = np.sum(mask)
                ndiscard = mbefore - mafter
                if ndiscard > 0:
                    print('{}:{} in {} {} artifact bins (={:5.2f}s) discarded'.\
                          format(rawname_,chn,int_type_cur,ndiscard,ndiscard/sfreq))
                chd = chdata[chni,mask] # noe that it is not modified
                
                dat_permod_perraw_perint[mod][rawname_][itcur][chni] = chd
                
                #if chd.size < 10:                       
                #chd = chdata[0,sl]
                r = np.quantile(chd,1-qch_hist_xshift)- np.quantile(chd,qch_hist_xshift)
                xshift = r * qmult 
                xshifts_rel_perint[itcur][chni] = max(xshifts_rel_perint[itcur][chni], xshift)
    xshifts_rel_perint_permod[mod] = xshifts_rel_perint
print('\nStats gather finished')

# Plot everything (takes time)

In [ ]:
rawnames_to_show = fnames_noext

#timerange = 0,100
#timerange = None
nr = len(rawnames_to_show); 
nc = len(int_types_templ)
#nr =2
ww = 10; hh = 3
nbins_hist = 100

qsh = 5e-2  # what will be used for limits computations
qsh_disp = 5e-3 # what will be given to hist function

subjindlist_str = ','.join(map(str,subjinds) )
for mod in modalities:
    fig,axs = plt.subplots(nrows=nr, ncols=nc, figsize= (nc*ww,nr*hh), sharex='col')
    plt.subplots_adjust(left=0.03,right=0.99, bottom=0.02,top=0.97)
    mns = nc*[np.inf]
    mxs = nc*[-np.inf]
    
    for i,rawname_ in enumerate(rawnames_to_show):
        rawind = i
        rawname_ = rawnames_to_show[rawind]
        raw = raws_permod_both_sides[rawname_][mod ]
        for j in range(nc):     
            ax = axs[i,j]
            itcur = int_types_templ[j]
            
            # it will be reset later if interval is indeed found
            ax.set_title('{} {} interval_type={}, 0s'.\
                         format(fnames_noext[rawind], mod, itcur) )

            
            chds = dat_permod_perraw_perint[mod][rawname_].get(itcur,None)
            if chds is None:
                continue
            if mod == 'MEG':
                chnames = np.array(raw.ch_names)[meg_chis]
            elif mod == 'src':
                chnames = np.array(raw.ch_names)[src_chis]
            else:
                chnames = raw.ch_names
            for chni,chn in enumerate(chnames):
                chdata, times = raw[chn]
                clr = cmap(vals[chni ])

                chd = chds[chni]

                # sum shifts over all prev channel indices
                xshift = np.sum( xshifts_rel_perint_permod[mod][itcur][:chni] )
                chd2 = chd + xshift
                if chd2.size == 0:
                    print('fdf')
                    continue

                q0 = np.quantile(chd2,qsh)
                q1 = np.quantile(chd2,1-qsh)
                q0_disp = np.quantile(chd2,qsh_disp)
                q1_disp = np.quantile(chd2,1-qsh_disp)

                ax.hist(chd2, bins=nbins_hist, label=chn, alpha = 0.5, 
                        range=(q0_disp,q1_disp), color=clr )
                ax.axvline(x=np.mean(chd2), c=clr, ls=':')

                mns[j] = min(mns[j], q0)
                mxs[j] = max(mxs[j], q1)
                
#                 if chn == 'LFPL12':
#                     print(mod,rawname_,itcur,'LFPL12',np.min(chd2),np.max(chd2))

                #print('{} shift {}  q = {},{}'.format(chn,xshift,q0,q1) )


            ax.legend(loc='upper left')
            ax.grid()
            ax.set_title('{} {} interval_type={}, {:.2f}s'.
                         format(fnames_noext[rawind], mod, itcur, len(chd2)/raw.info['sfreq'] ) )

        print('  {} of {} finished'.format(mod, rawname_))
    for i in range(nr):
        for j in range(nc):    
            if not np.any( np.isinf([mns[j], mxs[j] ] ) ):
                axs[i,j].set_xlim(mns[j],mxs[j])

    plt.savefig('{}_stat_across_subj_highpass{}_{}.pdf'.format(subjindlist_str,highpass,mod))
    plt.close()
    print('{} finished'.format(mod))

In [ ]:
if highpass == 2:
    szEMG = 5e-4
    szLFP = 1e-5
    defrange_permod = {'EMG':(-szEMG,szEMG), 'LFP':(-szLFP,szLFP) } # for 1 Hz fitlered data
else:
    rangeEMG = (0.00022087509714765474, 0.003293280061916448)
    rangeLFP = (-0.00016875398068805225, 0.0019851821198426474)
    defrange_permod = {'EMG':rangeEMG, 'LFP':rangeLFP}

In [ ]:
defrange_permod

In [ ]:
#defrange  = None

In [ ]:
#pp = ax.get_children()[0]

In [ ]:
cmap(vals[0])

In [ ]:
xshifts_rel_permod

In [ ]:
#timerange = 0,100
#timerange = None
nr = len(fnames_noext); nc=len(modalities)
#nr =2
ww = 10; hh = 3
fig,axs = plt.subplots(nrows=nr, ncols=nc, figsize= (nc*ww,nr*hh), sharex='col')
mns = nc*[np.inf]
mxs = nc*[-np.inf]

qsh = 5e-2
qmult = 1.15

xshifts_rel_permod = {}
for mod in modalities:
    n_channels = len( subraws[mod ][0].ch_names )
    xshifts_rel = [0]*n_channels
    xshifts_rel_permod[mod] = xshifts_rel

#xshifts0 = 0 * [nc]
    
# first gather info 
for i in range(nr):
    for j in range(nc):
        ax = axs[i,j]
        mod = modalities[j]
        raw = subraws[mod ][i]
        
        n_channels = len(raw.ch_names)
        for chni,chn in enumerate(raw.ch_names):
            chdata, times = raw[chn]
            if timerange is not None:
                sl = slice(i0,i1)
            else:
                sl = slice(None)
            chd = chdata[0,sl]
            xshift = ( np.quantile(chd,0.9)- np.quantile(chd,0.1) ) * qmult 
            
            xshifts_rel_permod[mod][chni] = max(xshifts_rel_permod[mod][chni], xshift)
        
for i in range(nr):
    for j in range(nc):     
        ax = axs[i,j]
        mod = modalities[j]
        raw = subraws[mod ][i]
        
        for chni,chn in enumerate(raw.ch_names):
            chdata, times = raw[chn]
            clr = cmap(vals[chni ])
            
            if timerange is not None:
                sl = slice(i0,i1)
            else:
                sl = slice(None)
            chd = chdata[0,sl]
            
            xshift = np.sum( xshifts_rel_permod[mod][:chni] )
            chd2 = chd + xshift
#             if defrange_permod[mod] is not None:
#                 rngshift = defrange_permod[mod] + xshift
#             else:
#                 rngshift = None
            
            q0 = np.quantile(chd2,qsh)
            q1 = np.quantile(chd2,1-qsh)
#             q0_ = q0 - np.abs(q0) * 2
#             q1_ = q1 + np.abs(q1) * 2
            q0_ = np.quantile(chd2,1e-2)
            q1_ = np.quantile(chd2,1-1e-2)
            
            ax.hist(chd2, bins=200, label=chn, alpha = 0.5, range=(q0_,q1_), color=clr )
            ax.axvline(x=np.mean(chd2), c=clr, ls=':')

            mns[j] = min(mns[j], q0)
            mxs[j] = max(mxs[j], q1)
            
            #print('{} shift {}  q = {},{}'.format(chn,xshift,q0,q1) )
            
            
        ax.legend(loc='upper left')
        ax.grid()
        ax.set_title('{}_{}_time={}'.format(fnames_noext[i], mod, timerange ) )
        
for i in range(nr):
    for j in range(nc):    
        axs[i,j].set_xlim(mns[j],mxs[j])
    
plt.savefig('across_subj_highpass{}.pdf'.format(highpass))

In [ ]:
defrange_permod[mod] 

In [ ]:
mns,mxs

In [ ]:
mns[1]*2, mxs[1]*2

In [ ]:
#help(plt.hist)

In [ ]:
np.quantile(chdata,0.98)

In [ ]:
int_types = ['trem_L', 'notrem_L', 'hold_L', 'move_L']

In [ ]:
fnames_noext

In [ ]:
ivalis_per_raw = {}
for fn in fnames_noext:
    ivalis = {'entire':[ times[0], times[-1], 'entire' ]}
    anns_fn = fn + '_anns.txt'
    anns_fn_full = os.path.join(data_dir, anns_fn)
    if os.path.exists(anns_fn_full):
        anns = mne.read_annotations(anns_fn_full)
        ivalis_ = utils.ann2ivalDict(anns)
        ivalis.update(ivalis_)
    ivalis_per_raw[fn] = ivalis

In [ ]:
for subj in gv.gen_subj_info:
    info = gv.gen_subj_info[subj]
    ts = info.get('tremor_side','UNDEF')
    ms = info.get('move_side','UNDEF')
    s = ''
    if ts != ms:
        s = '----'
    print(s,subj,ts,ms)

In [ ]:
gv.gen_subj_info['S04']['tremor_side'], gv.gen_subj_info['S04']['move_side']

In [ ]:
maintremside

In [ ]:
raw.info['sfreq']

In [ ]:
import globvars as gv
data_dir = gv.data_dir

In [ ]:
chd.shape

In [ ]:
#%debug


mn_permod = {}
mx_permod = {}
for mod in modalities:
    mn,mx = np.inf, -np.inf
    mn_permod[mod ] = mn
    mx_permod[mod ] = mx

subjinds_toshow = [1]
for subji in subjinds_toshow:  #subjinds:
    dessubj = 'S{:02d}'.format(subji)
        
    inds = []
    for ii in range(len(fnames_noext)):
        rawn = fnames_noext[ii]
        sind_str,medcond,task = utils.getParamsFromRawname(rawn)
        #print(sind_str)
        if sind_str == dessubj:
            inds += [ii]
    
    nr = len(inds); nc=len(modalities)*len(int_types)
    ww = 10; hh = 3
    fig,axs = plt.subplots(nrows=nr, ncols=nc, figsize= (nc*ww,nr*hh), sharex='col')
    plt.subplots_adjust(left=0.02,right=0.99)
    rowind = 0

    xshifts = []
    for j in range(nc):
        modind = j // len(int_types)
        
        mod = modalities[modind]
            
        for i in range(nr):
            ax = axs[i,j]
            
            fn = fnames_noext[inds[i]]
            raw = subraws[mod ][inds[i]]
#             print(i,j,fn)
#             continue
            
            ivalis = {'entire':[ times[0], times[-1], 'entire' ]}
            anns_fn = fn + '_anns.txt'
            anns_fn_full = os.path.join(data_dir, anns_fn)
            if os.path.exists(anns_fn_full):
                anns = mne.read_annotations(anns_fn_full)
                ivalis_ = utils.ann2ivalDict(anns)
                ivalis.update(ivalis_)


            int_type_cur = int_types[ j % len(int_types)  ]
            #for int_type_cur in int_types:
            for chni,chn in enumerate(raw.ch_names):
                chdata, times = raw[chn]

                #int_type_cur
                chds = []
                intervals = ivalis.get(int_type_cur, [None] )
                for timerange in intervals:
                    if timerange is not None:
                        i0, i1 = raw.time_as_index([timerange[0],timerange[1]])
                        sl = slice(i0,i1)
                    else:
                        print('{} Wrong interval type {}'.format(rawname_,int_type_cur) )
                        sl = np.arange( 2)
                    chd_ = chdata[0,sl]
                    chds += [chd_]
                chd = np.hstack(chds)
                if chd.size > 10:
                    xshift = np.quantile(chd,0.9)
                    chd2 = chd+xshift
                    
                    mn_,mx_ = np.quantile(chd2,qsh),  np.quantile(chd2,1-qsh)
                    mn_permod[mod ] = min( mn_permod[mod ], mn_ )
                    mx_permod[mod ] = max( mx_permod[mod ], mx_ )                
                    
                    clr = cmap(vals[chni ])
                    ax.hist(chd2, bins=50, label=chn, alpha = 0.6, range = defrange_permod[mod], color=clr)
                    ax.axvline(x=xshift + np.mean(chd), c=clr, ls=':')

                    print('{},{}; int_type={} min={:.3f}, max={:.3f}'.format(chn,int_type_cur,int_type_cur,mn_*10000,mx_*10000) )
            
            ax.legend(loc='upper left')
            ax.grid()
            if timerange is None:
                timerange = [-1,-1]
            ax.set_title('{}_{}_(last)time={:.2f},{:.2f}_totlen={:.2f}'.format(fn, int_type_cur, timerange[0],timerange[1], chd.size / raw.info['sfreq']  ) )
            
    for i in range(nr):
        for j in range(nc):
            modind = j // len(int_types)
            mod = modalities[modind]
            mn = mn_permod[mod ]
            mx = mx_permod[mod ]
            if not (np.isinf(mn) or np.isinf(mx) ):
                axs[i,j].set_xlim(mn,mx)

    plt.savefig('{}_highpass{}_stats_across_cond_tasks.pdf'.format(dessubj, highpass))
    plt.close()

In [ ]:
help(plt.hist)

In [ ]:
fnames_noext

In [ ]:
mn_permod

In [ ]:
mx_permod

In [ ]:
ivalis

In [ ]:
inds

In [ ]:
inds

In [ ]:
print(inds)

In [ ]:
help(plt.hist)